# 1. Loading the fMRI images

Let's first learn how to load fMRI images, and also understand the dimensions of fMRI images.

In [1]:
#main library for preprocessing fMRI images
import nibabel as nb
print(nb.__version__)

# utilities
from pathlib import Path            #combine path elements with /

3.2.1


Setting the path directory

In [2]:
data_dir = Path('data')

## 1. Importing data

### Loading

Load some data.  Note that the data is from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/29352  which contains 30 subjects data.  10 for AD (Alzheimers) (denoted nothing), 10 for MCI (Mild Cognitive Impairement) (denoted 1), and 10 for NCS (Normal) (denoted 2).  

Each of the subject has two information - functional and structual.

Let's try load the eighth (i just simply randomly choose) AD subject images.

In [3]:
t1w  = nb.load(data_dir /'subj_08_structural.nii')  #t1w stands for T1 weighted image
bold = nb.load(data_dir /'subj_08_functional.nii')  #blood oxygenization image (the functional MRI)
t1w